In [ ]:
!pip install stable-baselines3[extra]

!pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2
#!pip install pygame
!pip install pandas
!pip install stable-baselines[mpi]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 5.2 MB/s 
     |████████████████████████████████| 1.5 MB 47.1 MB/s 
     |████████████████████████████████| 1.6 MB 40.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616822 sha256=ccea87ba3cfb8a2aaed7474bd5d5749b2733f54894b758a231b07403b9f38876
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441027 sha256=f8103edd7d8a8aff284d3f10d5c3fd1b2eafa07bf4418b775749d42a43a4f5fc
  Stored in directory: /root/.cache/pip/wheels/87/67/2e/6147e7912fe37f5408b80d07527dab807c1d25f5c403a9538a
Successfully built gym AutoROM.accept-rom-license


In [ ]:
import pandas as pd
import datetime
from google.colab import drive
drive.mount('/content/drive')

#VPP_data = pd.read_csv('drive/My Drive/DAI-Labor/Thesis/VPP_table.csv')
VPP_data = pd.read_csv('drive/My Drive/DAI-Labor/Thesis/VPP_table_complete.csv')
VPP_data['time'] = pd.to_datetime(VPP_data['time'])
time_serie = VPP_data['time'].values
simulation_len = len(time_serie)
VPP_data = VPP_data.iloc[:,2:]

#VPP_data.info()
VPP_data.head()

Mounted at /content/drive


,ev_power,household_power,renewable_power,EUR/kWh,total_load,total_cost,Available_energy_sources,1,2,3,...,69,70,71,72,73,74,75,76,77,78
0,0.0,0.403179,0.238,0.04084,0.165179,0.001686,[],0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.405153,0.238,0.04084,0.167153,0.001707,[],0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.7,0.537645,0.238,0.04084,3.999645,0.040836,"[{'vehicle_id': 1, 'energy': 51.530815548270304}]",51.530816,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.7,0.779943,0.238,0.04084,4.241943,0.043310,"[{'vehicle_id': 1, 'energy': 52.4558155482703}]",52.455816,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.7,0.732574,0.223,0.04115,4.209574,0.043306,"[{'vehicle_id': 1, 'energy': 53.380815548270306}]",53.380816,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# import gym
# #import pygame

In [2]:
from gym import Env
from gym.spaces import Discrete, Box, Dict
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
%matplotlib inline
#from IPython import display
#export DISPLAY=localhost:0.0
import stable_baselines3 as sb3
sb3.get_system_info()

In [ ]:
Init_space_ev_power = np.zeros(1,dtype=np.float32)
Init_space_total_load = np.zeros(1,dtype=np.float32)
Init_space_total_cost = np.zeros(1,dtype=np.float32)

Init_space_ev_power[0] = VPP_data["ev_power"].values[0]
Init_space_total_load[0] = VPP_data["total_load"].values[0]
Init_space_total_cost[0] = VPP_data["total_cost"].values[0]

LIST_Init_space_Available_energy_sources = VPP_data["Available_energy_sources"].values
Unoptimized_Simulation_energy_State_space = np.float32(VPP_data.iloc[:,7:].values)
VPP_energies = Unoptimized_Simulation_energy_State_space

Init_space_Available_energy_sources = Unoptimized_Simulation_energy_State_space[0]
EV_simulation_n =len(Unoptimized_Simulation_energy_State_space[0])

print(len(Unoptimized_Simulation_energy_State_space))
print(EV_simulation_n)
print(Init_space_ev_power)
print(Init_space_total_load)
print(Init_space_total_cost)
print(Init_space_Available_energy_sources)

2881
78
[0.]
[0.165179]
[0.00168648]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]


In [ ]:
Init_space = {
    'ev_power': Init_space_ev_power, #EV load range in kW, depends on the infrastructure of the simulation
    'total_load': Init_space_total_load, #Total load range in kW, depends on the household and EV infrastructure of the simulation
    'total_cost': Init_space_total_cost, #Total cost (EUR) per timestep (15 min)
    'Available_energy_sources': Init_space_Available_energy_sources #all EV energy state
}

In [ ]:
class VPPEnv(Env):
  

  def __init__(self):
    #Actions we can take
    self.action_space = Box(low=-11, high=11, shape=(EV_simulation_n,), dtype=np.float32) #Array of continuous Power values to be charged to each vehicle in the simulation

    # Define constants for clearer code
    
    self.IDLE = 0
    self.CHARGE_3_7 = 3.7
    self.DISCHARGE_3_7 = -3.7
    self.CHARGE_7_4 = 7.4
    self.DISCHARGE_7_4 = -7.4
    self.CHARGE_11 = 11
    self.DISCHARGE_11 = -11

    #Charging station Power array
    #self.observation_space = Box(low = np.array([0]), high = np.array([1000])) #Example

    #TO DO: NORMALIZE ACTION SPACE (not if you use algorithm TD3 and DDPG)
    spaces = {
      'ev_power': Box(low=0, high=18.5, shape=(1,), dtype=np.float32), #EV load range in kW, depends on the infrastructure of the simulation
      # #'ev_power': gym.spaces.Box(low=0, high=18.5, shape=(simulation_len,)),
      'total_load': Box(low=-30, high=30, shape=(1,), dtype=np.float32), #Total load range in kW, depends on the household and EV infrastructure of the simulation
      'total_cost': Box(low=-0.30, high=0.60, shape=(1,), dtype=np.float32),#Total cost (EUR) per timestep (15 min)
      'Available_energy_sources': Box(low=0.0, high=100, shape=(EV_simulation_n,), dtype=np.float32)
      }
    #gym.spaces.Dict()
    dict_space = Dict(spaces)
    self.observation_space = dict_space

    #Set starting cond.
    self.state = Init_space
    #Set VPP session length
    self.VPP_energies = Unoptimized_Simulation_energy_State_space
    self.vpp_length = simulation_len

  def step(self, action):
    #evaluate step
    step = simulation_len - self.vpp_length

    #Apply action
    #self.state += action -1
    
    Energy_sources_t = self.VPP_energies[step]
    if step > 0:
      Energy_sources_t_1 = self.VPP_energies[step-1]
      #OR
      #Energy_sources_t_1 = self.state['Available_energy_sources']
    
    #Reward Initialization
    reward = 0

    #__INIT__ ACTION VECTOR LOOP
    for j in range(EV_simulation_n):
      #1. CHECK CURRENT STEP (case: current step in range)
      if Energy_sources_t[j] > 0 and Energy_sources_t[j] <= 100:

        if step == 0: #2.FIRST STEP CASE
          #4.1 ACTIONS:
          if action[j] == self.CHARGE_3_7: #(already considered charged)
            Energy_sources_t[j] = Energy_sources_t[j] #STORING ENERGY
          elif action[j] == self.CHARGE_7_4: #(already considered charged)
            Energy_sources_t[j] = Energy_sources_t[j] #STORING ENERGY
          elif action[j] == self.CHARGE_11: #(already considered charged)
            Energy_sources_t[j] = Energy_sources_t[j] #STORING ENERGY
          elif action[j] == self.IDLE: #(subtracting energy already considered charged)
            Energy_sources_t[j] = Energy_sources_t[j] - (3.7 * 0.25) #keep energy constant
            if Energy_sources_t[j] < 0: #Reached min capacity (kWh)
              Energy_sources_t[j] = 0
          elif action[j] == self.DISCHARGE_3_7: # (subtracting energy already considered charged AND pushed)
            Energy_sources_t[j] = Energy_sources_t[j] - 2*(3.7 * 0.25) #PUSHING ENERGY
            if Energy_sources_t[j] < 0: #Reached min capacity (kWh)
              Energy_sources_t[j] = 0
          elif action[j] == self.DISCHARGE_7_4: # (subtracting energy already considered charged AND pushed)
            Energy_sources_t[j] = Energy_sources_t[j] - (3.7 * 0.25) - (7.4 * 0.25) #PUSHING ENERGY
            if Energy_sources_t[j] < 0: #Reached min capacity (kWh)
              Energy_sources_t[j] = 0
          elif action[j] == self.DISCHARGE_11: # (subtracting energy already considered charged AND pushed)
            Energy_sources_t[j] = Energy_sources_t[j] - (3.7 * 0.25) - (11 * 0.25) #PUSHING ENERGY
            if Energy_sources_t[j] < 0: #Reached min capacity (kWh)
              Energy_sources_t[j] = 0
          else:
            raise ValueError("Received invalid action={} which is not part of the action space".format(action))

        else:#2.OTHER STEPS CASES > 0
          
          #3.Check previous step range (CASE 3.1: CAR ALREADY CONNECTED)
          if Energy_sources_t_1[j] > 0 and Energy_sources_t_1[j] <= 100:
            
            #4.2 ACTIONS:
            if action[j] == self.CHARGE_3_7:
              Energy_sources_t[j] = Energy_sources_t_1[j] + (3.7 * 0.25) #3.7 kW * 15 min = kWh STORING ENERGY
              if Energy_sources_t[j] > 100: #Reached max capacity (kWh)
                Energy_sources_t[j] = 100
            elif action[j] == self.CHARGE_7_4:
              Energy_sources_t[j] = Energy_sources_t_1[j] + (7.4 * 0.25) #7.4 kW * 15 min = kWh STORING ENERGY
              if Energy_sources_t[j] > 100: #Reached max capacity (kWh)
                Energy_sources_t[j] = 100
            elif action[j] == self.CHARGE_11:
              Energy_sources_t[j] = Energy_sources_t_1[j] + (11 * 0.25) #11 kW * 15 min = kWh STORING ENERGY
              if Energy_sources_t[j] > 100: #Reached max capacity (kWh)
                Energy_sources_t[j] = 100
            elif action[j] == self.IDLE:
              Energy_sources_t[j] = Energy_sources_t_1[j] #keep energy constant
            elif action[j] == self.DISCHARGE_3_7:
              Energy_sources_t[j] = Energy_sources_t_1[j] - (3.7 * 0.25) #3.7 kW * 15 min = kWh PUSHING ENERGY
              if Energy_sources_t[j] < 0: #Reached min capacity (kWh)
                Energy_sources_t[j] = 0
            elif action[j] == self.DISCHARGE_7_4:
              Energy_sources_t[j] = Energy_sources_t_1[j] - (7.4 * 0.25) #7.4 kW * 15 min = kWh PUSHING ENERGY
              if Energy_sources_t[j] < 0: #Reached min capacity (kWh)
                Energy_sources_t[j] = 0
            elif action[j] == self.DISCHARGE_11:
              Energy_sources_t[j] = Energy_sources_t_1[j] - (11 * 0.25) #3.7 kW * 15 min = kWh PUSHING ENERGY
              if Energy_sources_t[j] < 0: #Reached min capacity (kWh)
                Energy_sources_t[j] = 0
            else:
              raise ValueError("Received invalid action={} which is not part of the action space".format(action))
          
          #3.Check previous step (CASE 3.2: CAR JUST CONNECTED)
          elif Energy_sources_t_1[j] == 0: #When car just connects
            
            #4.3 ACTIONS:
            if action[j] == self.CHARGE_3_7: #(already considered charged)
              Energy_sources_t[j] = Energy_sources_t[j] #STORING ENERGY
            elif action[j] == self.CHARGE_7_4: #(already considered charged)
              Energy_sources_t[j] = Energy_sources_t[j] + ((7.4 * 0.25) - (3.7 * 0.25)) #7.4 kW * 15 min = kWh PUSHING ENERGY - the already considered pushed one
              if Energy_sources_t[j] > 100: #Reached max capacity (kWh)
                Energy_sources_t[j] = 100
            elif action[j] == self.CHARGE_11: #(already considered charged)
              Energy_sources_t[j] = Energy_sources_t[j] + ((11 * 0.25) - (3.7 * 0.25)) #11 kW * 15 min = kWh PUSHING ENERGY - the already considered pushed one
              if Energy_sources_t[j] > 100: #Reached max capacity (kWh)
                Energy_sources_t[j] = 100
            elif action[j] == self.IDLE: #(subtracting energy already considered charged)
              Energy_sources_t[j] = Energy_sources_t[j] - (3.7 * 0.25) #keep energy constant
              if Energy_sources_t[j] < 0: #Reached min capacity (kWh)
                Energy_sources_t[j] = 0
            elif action[j] == self.DISCHARGE_3_7: # (subtracting energy already considered charged AND pushed)
              Energy_sources_t[j] = Energy_sources_t[j] - 2*(3.7 * 0.25) #PUSHING ENERGY
              if Energy_sources_t[j] < 0: #Reached min capacity (kWh)
                Energy_sources_t[j] = 0
            elif action[j] == self.DISCHARGE_7_4: # (subtracting energy already considered charged AND pushed)
              Energy_sources_t[j] = Energy_sources_t[j] - (3.7 * 0.25) - (7.4 * 0.25) #PUSHING ENERGY
              if Energy_sources_t[j] < 0: #Reached min capacity (kWh)
                Energy_sources_t[j] = 0
            elif action[j] == self.DISCHARGE_11: # (subtracting energy already considered charged AND pushed)
              Energy_sources_t[j] = Energy_sources_t[j] - (3.7 * 0.25) - (11 * 0.25) #PUSHING ENERGY
              if Energy_sources_t[j] < 0: #Reached min capacity (kWh)
                Energy_sources_t[j] = 0
            else:
              raise ValueError("Received invalid action={} which is not part of the action space".format(action))
          
          #3.Check previous step (CASE 3.3: error previous state)
          else:
            raise ValueError("Available_energy_sources table out of ranges: state={} which is not part of the state space".format(Energy_sources_t_1))
      
      #1. CHECK CURRENT STEP (case: current step out of range)
      elif Energy_sources_t[j] != 0:
          raise ValueError("Available_energy_sources table out of ranges: state={} which is not part of the state space".format(Energy_sources_t))
    
      #Calculating reward for generated state j
      #TO DO: EVALUATE REWARD AT LEAVING TIME
      if Energy_sources_t[j] != 0:
        
        if Energy_sources_t[j] > 0 and Energy_sources_t[j] <= 25:
          reward += -2
        elif Energy_sources_t[j] > 25 and Energy_sources_t[j] <= 40:
          reward += -1
        elif Energy_sources_t[j] > 40 and Energy_sources_t[j] <= 55:
          reward += 0
        elif Energy_sources_t[j] > 55 and Energy_sources_t[j] <= 65:
          reward += 1
        elif Energy_sources_t[j] > 65 and Energy_sources_t[j] <= 85:
          reward += 2
        elif Energy_sources_t[j] > 85 and Energy_sources_t[j] < 95:
          reward += 1
        elif Energy_sources_t[j] > 85 and Energy_sources_t[j] < 99:
          reward += 0
        elif Energy_sources_t[j] >= 99 and Energy_sources_t[j] <= 100:
          reward += -1
    
    #__END__ ACTION VECTOR LOOP

    #VPP Table UPDATE
    self.VPP_energies[step] = Energy_sources_t
    #STATE UPDATE
    #self.state['Available_energy_sources'][step] = Energy_sources_t
    self.state['Available_energy_sources'] = Energy_sources_t


    #Reduce VPP session length by 1 step
    self.vpp_length -= 1
    #Check if VPP is done
    if self.vpp_length <= 0:
      done = True
    else:
      done = False

    #set placeholder for info
    info = {}

    #return step information
    return self.state, reward, done, info
  
  def render(self, mode = 'human'):
    #implement visualization
    pass

  def reset(self):
    #reset vpp power
    self.state = Init_space
    self.VPP_energies = Unoptimized_Simulation_energy_State_space
    #reset vpp session time
    self.vpp_length = len(time_serie)
    return self.state



In [ ]:
env = VPPEnv()

In [ ]:
from stable_baselines3.common.env_checker import check_env

# It will check your custom environment and output additional warnings if needed
check_env(env)

In [ ]:
env.action_space.sample()

3

In [ ]:
env.observation_space.sample()

OrderedDict([('Available_energy_sources',
              array([62.884724 , 53.32344  , 82.98895  , 42.479393 , 15.583014 ,
                     26.600084 , 70.824875 , 65.43931  , 95.48068  , 28.114069 ,
                     28.824974 , 12.379596 , 82.45935  , 79.66634  , 74.66252  ,
                     24.64583  , 64.25571  , 27.75163  ,  5.3394613, 44.489147 ,
                     54.424862 , 16.745892 , 49.59526  , 57.458885 , 61.21654  ,
                     73.53794  , 67.39286  , 30.640945 , 79.43672  , 36.899353 ,
                      4.346202 , 13.222768 ,  8.072905 , 66.688385 , 83.87428  ,
                      3.2074933, 98.52619  , 36.98965  , 20.533571 , 81.0088   ,
                     51.661816 , 18.182571 , 17.351511 , 98.185036 , 90.05553  ,
                      6.143688 , 68.93868  , 97.47169  , 47.752174 , 82.45042  ,
                     17.2562   , 78.83995  , 20.359802 , 49.983963 , 33.41187  ,
                      8.432486 , 84.33016  , 36.734398 , 31.481218 

In [ ]:
"""env = gym.make('CartPole-v1')"""
#states = env.observation_space.shape[0]
actions = env.action_space.n

In [ ]:
actions

7

In [ ]:
# episodes = 10
# for episode in range(1, episodes+1):
#     state = env.reset()
#     done = False
#     score = 0 

#     #img = plt.imshow(env.render(mode='rgb_array')) # only call this once


#     while not done:
#         #env.render()

#         action = np.ones(EV_simulation_n)
#         #action = random.choice([0,1])
#         for j in range(EV_simulation_n):
#           action[j] = env.action_space.sample()
        
#         n_state, reward, done, info = env.step(action)
#         score+=reward
#     print('Episode:{} Score:{}'.format(episode, score))
#     VPP_energies = pd.DataFrame(env.VPP_energies)
#     VPP_energies.plot(legend = None)

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO, A2C
#Make your storing directories
log_path = 'drive/My Drive/DAI-Labor/Thesis/logs_model/'

#Vectorized environment wrapper
Vec_env = DummyVecEnv([lambda: env])
#PPO??
#model  = PPO('MlpPolicy', Vec_env, verbose=1, tensorboard_log=log_path)
#A2C??
model = A2C("MultiInputPolicy", Vec_env, verbose=1, tensorboard_log=log_path)


Using cpu device


In [ ]:
model.learn(total_timesteps=50000)

Logging to drive/My Drive/DAI-Labor/Thesis/logs_model/A2C_1


IndexError: ignored

2. Create a Deep Learning Model with Keras

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
#tf.config.list_physical_devices('GPU')

In [ ]:
#Definition of states

states =  env.observation_space['Available_energy_sources'].shape + env.observation_space['ev_power'].shape + env.observation_space['total_load'].shape + env.observation_space['total_cost'].shape
print(states)
#actions = env.action_space.n

In [ ]:
env.observation_space.sample()

In [ ]:
def build_model(states, actions):
    model = Dense()
    #model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu', input_shape = states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(1, activation='relu'))
    #model.add(Dense(1, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
#del model

In [ ]:
model = build_model(states, actions)
#model = A2C('CnnPolicy', env).learn(total_timesteps=1000)
model.summary()

In [ ]:
#model = build_model(states, actions)

#TO DO: CUSTOM OPTIMIZATION MODEL 
#model = A2C('CnnPolicy', env).learn(total_timesteps=1000)
#model.summary()

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate = 1e-3), metrics=['mse'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)